### Let's build GPT: from scratch, in code, spelled out.

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open("/Users/nihaljayanth/Development/makemore/data/input.txt", "r") as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
chars = sorted(set(list(text)))
vocab_size = len(chars)
print(f"Number of unique charactes in this dataset: {vocab_size},\nAnd they are: {''.join(chars)}")

Number of unique charactes in this dataset: 65,
And they are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
stoi = {c:idx for idx, c in enumerate(chars)}
itos = {idx:c for idx, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
print(encode("Hii there"))
print(decode([20, 47, 47, 1, 58, 46, 43, 56, 43]))
print(decode(encode("Hii there")))

[20, 47, 47, 1, 58, 46, 43, 56, 43]
Hii there
Hii there


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)

torch.Size([1115394])


In [8]:
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]

In [9]:
block_size = 8

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    inp = x[:i+1]
    pred = y[i]
    print(f"when the input is {inp} the target is {pred}")

when the input is tensor([18]) the target is 47
when the input is tensor([18, 47]) the target is 56
when the input is tensor([18, 47, 56]) the target is 57
when the input is tensor([18, 47, 56, 57]) the target is 58
when the input is tensor([18, 47, 56, 57, 58]) the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    sample = torch.randint(high=len(data)-block_size, size=(batch_size,))
    x = torch.stack([data[s:s+block_size] for s in sample])
    y = torch.stack([data[s+1:s+block_size+1] for s in sample])
    return x, y

In [12]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [13]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        # B = batch_size, T = seq_len, C = output_classes
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
lm = BigramLanguageModel(vocab_size=vocab_size)

In [15]:
logits, loss = lm(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
decode(lm.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=10)[0].tolist())

'\nSKIcLT;AcE'

In [17]:
optimizer = torch.optim.AdamW(lm.parameters(), lr=1e-3)

In [31]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch("train")
    
    logits, loss = lm(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.474673271179199


In [32]:
for param in nn.Linear(2, 1).parameters():
    print(param)

Parameter containing:
tensor([[0.5579, 0.0914]], requires_grad=True)
Parameter containing:
tensor([0.6675], requires_grad=True)


In [33]:
print(decode(lm.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


N hby, m f rshaireik, f
Thimose nd

OForoly, mm?
ATIt, senore manatheverintthtycheelllle-he or ho l thric ld weito.
Pr toou
USIINor wis d; now thored lr co, gretordar, m b; by t ancon dshas isenyodre du terere'erds.'d I ITiny pout hen aved youa ty lorvert or p n s bliofe y,
GLAnd tht, tenel teave beslas, f bar weor the widy t aveauraghoo malo mesthe fif tofake It ind, r nd man h ate,
My, tht ari'se theruthis, w
DY s d acthithatok Be pleresf ak


Aseile,
Waf f pompo ind; he, m t LUCourigure y ord
